In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import my_data_frequency_conversor as dfc
import my_models as mm
import my_llm_ts_preprocessor as tsp
import my_prompting_tool as mpt 

from statsmodels.tsa.holtwinters import SimpleExpSmoothing 
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import Holt

from nixtla import NixtlaClient
from dotenv import load_dotenv
import os

load_dotenv()  # Carga el .env

from gluonts.dataset.pandas import PandasDataset

from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split


In [ ]:
DATA_HOURS = 'DATA/ETHUSD-H1.csv'
DATA_DAYS = 'DATA/ETHUSD-D1.csv'
DATA_MONTHS = 'DATA/ETHUSD-Monthly.csv'
arima_h = mm.Model('ARIMA', 0, 0, 0)
arima_d = mm.Model('ARIMA-d', 0, 0, 0)
arima_m = mm.Model('ARIMA-m', 0, 0, 0)
exp_smooth_h = mm.Model('EXPONENTIAL SMOOTHING', 0, 0, 0)
exp_smooth_d = mm.Model('EXPONENTIAL SMOOTHING', 0, 0, 0)
exp_smooth_m = mm.Model('EXPONENTIAL SMOOTHING', 0, 0, 0)
holt_h = mm.Model('HOLT', 0, 0, 0)
holt_d = mm.Model('HOLT', 0, 0, 0)
holt_m = mm.Model('HOLT', 0, 0, 0)
holt_winters_h = mm.Model('HOLT WINTERS', 0, 0, 0)
holt_winters_d = mm.Model('HOLT WINTERS', 0, 0, 0)
holt_winters_m = mm.Model('HOLT WINTERS', 0, 0, 0)
timegpt_h = mm.Model('TIMEGPT', 0, 0, 0)
timegpt_d = mm.Model('TIMEGPT', 0, 0, 0)
timegpt_m = mm.Model('TIMEGPT', 0, 0, 0)
lagllama_h = mm.Model('LAGLLAMA', 0, 0, 0)
lagllama_d = mm.Model('LAGLLAMA', 0, 0, 0)
lagllama_m = mm.Model('LAGLLAMA', 0, 0, 0)
gpt_h = mm.Model('GPT-H', 0, 0, 0)
gpt_d = mm.Model('GPT-D', 0, 0, 0)
gpt_m = mm.Model('GPT-M', 0, 0, 0)


In [ ]:
df = pd.read_csv(DATA_HOURS, delimiter=',')
train_data_hours, val_data_hours = dfc.split_data(df)
train_data_hours = train_data_hours.asfreq('h')
val_data_hours = val_data_hours.asfreq('h')
print(train_data_hours.head())
print(val_data_hours.head())

In [ ]:
# Crear una serie de ejemplo
series = [0.123, 1.23, 12.3, 123.0, 1000.0]

# Instanciar el preprocesador
preprocessor = tsp.TimeSeriesPreprocessor(percentile=95)

# Procesar la serie
processed_series = preprocessor.preprocess(series)

print(processed_series)


In [ ]:
df = pd.read_csv(DATA_DAYS, delimiter=',')
train_data_days, val_data_days = dfc.split_data(df)
train_data_days = train_data_days.asfreq('d')
val_data_days = val_data_days.asfreq('d')
print(train_data_days.head())
print(val_data_days.head())

In [ ]:
data_months = pd.read_csv(DATA_MONTHS, delimiter=',')
train_data_months, val_data_months = dfc.split_data(data_months)
train_data_months = train_data_months.asfreq('MS')
val_data_months = val_data_months.asfreq('MS')
print(train_data_months.head())
print(val_data_months.head())

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

In [ ]:
# Convertir la serie en DataFrame con las columnas correctas
arima_m.nixtla_format = dfc.convert_to_nixtla(train_data_months)
print(arima_m.nixtla_format.head())
arima_m.model = StatsForecast(
    models=[AutoARIMA(season_length=1)],
    freq='MS'
)

arima_m.model.fit(arima_m.nixtla_format)
arima_m.forecast = arima_m.model.predict(h=12, level=[95])
print(arima_m.forecast)

In [ ]:
# Plot ARIMA forecast against the actual data with the confidence intervals
train_df = train_data_months.reset_index().iloc[:, :2]  # Solo tomar las dos primeras columnas
train_df.columns = ['ds', 'y']
train_df['type'] = 'Train'

val_df = val_data_months.reset_index().iloc[:, :2]  # Solo tomar las dos primeras columnas
val_df.columns = ['ds', 'y']  # Renombrar columnas
val_df['type'] = 'Validation'

# Concatenar los datos de entrenamiento y validación para graficar
all_data = pd.concat([train_df, val_df])

# Graficar los datos de entrenamiento y validación
plt.figure(figsize=(12, 6))
plt.plot(all_data['ds'], all_data['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
plt.plot(arima_m.forecast['ds'], arima_m.forecast['AutoARIMA'], label='Predicciones', color='red')

# Graficar el intervalo de confianza
plt.fill_between(
    arima_m.forecast['ds'],
    arima_m.forecast['AutoARIMA-lo-95'],
    arima_m.forecast['AutoARIMA-hi-95'],
    color='pink',
    alpha=0.3,
    label='Intervalo de Confianza 95%'
)

# Añadir leyenda y etiquetas
plt.legend()
plt.title('Predicciones vs Datos Reales')
plt.xlabel('Fecha')
plt.ylabel('Open')

# Mostrar la gráfica
plt.show()

In [ ]:
arima_d.nixtla_format = dfc.convert_to_nixtla(train_data_days)
print(arima_d.nixtla_format.head())
arima_d.model = StatsForecast(
    models=[AutoARIMA(season_length=1)],
    freq='D'
)
'''
season_length=7:Se asume que hay un patrón semanal en los datos diarios. 
Si no es el caso, puedes cambiarlo a 1 (sin estacionalidad) o 365 (estacionalidad anual).
'''
arima_d.model.fit(arima_d.nixtla_format)
arima_d.forecast = arima_d.model.predict(h=366, level=[95])
print(arima_d.forecast)

In [ ]:
# Plot ARIMA forecast against the actual data with the confidence intervals
train_df = train_data_days.reset_index().iloc[:, :2]  # Solo tomar las dos primeras columnas
train_df.columns = ['ds', 'y']
train_df['type'] = 'Train'

val_df = val_data_days.reset_index().iloc[:, :2]  # Solo tomar las dos primeras columnas
val_df.columns = ['ds', 'y']  # Renombrar columnas
val_df['type'] = 'Validation'

# Concatenar los datos de entrenamiento y validación para graficar
all_data = pd.concat([train_df, val_df])

# Graficar los datos de entrenamiento y validación
plt.figure(figsize=(12, 6))
plt.plot(all_data['ds'], all_data['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
plt.plot(arima_d.forecast['ds'], arima_d.forecast['AutoARIMA'], label='Predicciones', color='red')

# Graficar el intervalo de confianza
plt.fill_between(
    arima_d.forecast['ds'],
    arima_d.forecast['AutoARIMA-lo-95'],
    arima_d.forecast['AutoARIMA-hi-95'],
    color='pink',
    alpha=0.3,
    label='Intervalo de Confianza 95%'
)

# Añadir leyenda y etiquetas
plt.legend()
plt.title('Predicciones vs Datos Reales')
plt.xlabel('Fecha')
plt.ylabel('Open')

# Mostrar la gráfica
plt.show()

In [ ]:
arima_h.nixtla_format = dfc.convert_to_nixtla(train_data_hours)
print(arima_h.nixtla_format.head())
arima_h.model = StatsForecast(
    models=[AutoARIMA(season_length=24)],
    freq='H'
)
'''
season_length=24: Se asume que hay un patrón diario en los datos horarios.
Si no es el caso, puedes cambiarlo a 1 (sin estacionalidad) o 168 (estacionalidad semanal).
'''
arima_h.model.fit(arima_h.nixtla_format)
arima_h.forecast = arima_h.model.predict(h=24*366, level=[95])
print(arima_h.forecast.head())

In [ ]:
# Plot ARIMA forecast against the actual data with the confidence intervals
train_df = train_data_hours.reset_index().iloc[:, :2]  # Solo tomar las dos primeras columnas
train_df.columns = ['ds', 'y']
train_df['type'] = 'Train'

val_df = val_data_hours.reset_index().iloc[:, :2]  # Solo tomar las dos primeras columnas
val_df.columns = ['ds', 'y']  # Renombrar columnas
val_df['type'] = 'Validation'

# Concatenar los datos de entrenamiento y validación para graficar
all_data = pd.concat([train_df, val_df])

# Graficar los datos de entrenamiento y validación
plt.figure(figsize=(12, 6))
plt.plot(all_data['ds'], all_data['y'], label='Datos Reales', color='blue')

# Graficar las predicciones
plt.plot(arima_h.forecast['ds'], arima_h.forecast['AutoARIMA'], label='Predicciones', color='red')

# Graficar el intervalo de confianza
plt.fill_between(
    arima_h.forecast['ds'],
    arima_h.forecast['AutoARIMA-lo-95'],
    arima_h.forecast['AutoARIMA-hi-95'],
    color='pink',
    alpha=0.3,
    label='Intervalo de Confianza 95%'
)

# Añadir leyenda y etiquetas
plt.legend()
plt.title('Predicciones vs Datos Reales')
plt.xlabel('Fecha')
plt.ylabel('Open')

# Mostrar la gráfica
plt.show()

In [ ]:
# Ajustar el modelo con cálculo óptimo de alpha
exp_smooth_m.model = SimpleExpSmoothing(train_data_months).fit(optimized=True)

# Obtener el alpha óptimo
exp_smooth_m.optimal_alpha = exp_smooth_m.model.model.params["smoothing_level"]
print(f"Alpha óptimo: {exp_smooth_m.optimal_alpha:.4f}")

# Predicción a futuro (próximo año)
exp_smooth_m.forecast = exp_smooth_m.model.forecast(steps=12)
exp_smooth_m.forecast


In [ ]:
# Ajustar el modelo con cálculo óptimo de alpha
exp_smooth_d.model = SimpleExpSmoothing(train_data_days).fit(optimized=True)

# Obtener el alpha óptimo
exp_smooth_d.optimal_alpha = exp_smooth_d.model.model.params["smoothing_level"]
print(f"Alpha óptimo: {exp_smooth_d.optimal_alpha:.4f}")

# Predicción a futuro (próximo año)
exp_smooth_d.forecast = exp_smooth_d.model.forecast(steps=366)
exp_smooth_d.forecast

In [ ]:
# Ajustar el modelo con cálculo óptimo de alpha
exp_smooth_h.model = SimpleExpSmoothing(train_data_hours).fit(optimized=True)

# Obtener el alpha óptimo
exp_smooth_h.optimal_alpha = exp_smooth_h.model.model.params["smoothing_level"]
print(f"Alpha óptimo: {exp_smooth_h.optimal_alpha:.4f}")

# Predicción a futuro (próximo año)
exp_smooth_h.forecast = exp_smooth_h.model.forecast(steps=366*24)
exp_smooth_h.forecast

In [ ]:
holt_m.model = Holt(train_data_months).fit(optimized=True)
holt_m.optimal_alpha = holt_m.model.model.params["smoothing_level"]
holt_m.optimal_beta = holt_m.model.model.params["smoothing_trend"]
print(f"Alpha óptimo: {holt_m.optimal_alpha:.4f}")
print(f"Beta óptimo: {holt_m.optimal_beta:.4f}")
holt_m.forecast = holt_m.model.forecast(steps=12)
holt_m.forecast

In [ ]:
holt_d.model = Holt(train_data_days).fit(optimized=True)
holt_d.optimal_alpha = holt_d.model.model.params["smoothing_level"]
holt_d.optimal_beta = holt_d.model.model.params["smoothing_trend"]
print(f"Alpha óptimo: {holt_d.optimal_alpha:.4f}")
print(f"Beta óptimo: {holt_d.optimal_beta:.4f}")
holt_d.forecast = holt_d.model.forecast(steps=366)
holt_d.forecast

In [ ]:
holt_h.model = Holt(train_data_hours).fit(optimized=True)
holt_h.optimal_alpha = holt_h.model.model.params["smoothing_level"]
holt_h.optimal_beta = holt_h.model.model.params["smoothing_trend"]
print(f"Alpha óptimo: {holt_h.optimal_alpha:.4f}")
print(f"Beta óptimo: {holt_h.optimal_beta:.4f}")
holt_h.forecast = holt_h.model.forecast(steps=366*24)
holt_h.forecast

In [ ]:
# Ajustar el modelo con cálculo óptimo de alpha
holt_winters_m.model = ExponentialSmoothing(train_data_months, trend='add', seasonal='add', seasonal_periods=12).fit(optimized=True)

# Obtener el alpha óptimo
holt_winters_m.optimal_params = holt_winters_m.model.params
print(f"Parámetros óptimos: {holt_winters_m.optimal_params}")

# Predicción a futuro (próximo año)
holt_winters_m.forecast = holt_winters_m.model.forecast(steps=12)
holt_winters_m.forecast

In [ ]:
# Ajustar el modelo con cálculo óptimo de alpha, seasonal period de 7 para intentar calcular una ciclidad semanal
holt_winters_d.model = ExponentialSmoothing(train_data_days, trend='add', seasonal='add', seasonal_periods=7).fit(optimized=True)

# Obtener el alpha óptimo
holt_winters_d.optimal_params = holt_winters_d.model.params
print(f"Parámetros óptimos: {holt_winters_d.optimal_params}")

# Predicción a futuro (próximo año)
holt_winters_d.forecast = holt_winters_d.model.forecast(steps=366)
holt_winters_d.forecast


In [ ]:
# Ajustar el modelo con cálculo óptimo de alpha, seasonal period de 24 para intentar calcular una ciclidad diaria
holt_winters_h.model = ExponentialSmoothing(train_data_hours, trend='add', seasonal='add', seasonal_periods=24).fit(optimized=True)

# Obtener el alpha óptimo
holt_winters_h.optimal_params = holt_winters_h.model.params
print(f"Parámetros óptimos: {holt_winters_h.optimal_params}")

# Predicción a futuro (próximo año)
holt_winters_h.forecast = holt_winters_h.model.forecast(steps=366*24)
holt_winters_h.forecast

In [ ]:
my_api_key = os.getenv("MY_NIXTLA_API_KEY")

In [ ]:
nixtla_client = NixtlaClient(
    api_key = my_api_key
)
nixtla_client.validate_api_key()

timegpt_m.predictions = nixtla_client.predict(
    df=train_data_months,
    horizon=12,
    time_col='DateTime',
    target_col='Open',
)

timegpt_d.predictions = nixtla_client.predict(
    df=train_data_days,
    horizon=366,
    time_col='DateTime',
    target_col='Open',
)

timegpt_h.predictions = nixtla_client.predict(
    df=train_data_hours,
    horizon=366*24,
    time_col='DateTime',
    target_col='Open',
)


In [ ]:
#!git clone https://github.com/SalesforceAIResearch/uni2ts/
%pip install uni2ts



In [ ]:
from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule

In [ ]:

SIZE = "small"  # model size: choose from {'small', 'base', 'large'}
PDT = 20  # prediction length: any positive integer
CTX = 200  # context length: any positive integer
PSZ = "auto"  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = 32  # batch size: any positive integer
TEST = 100  # test set length: any positive integer

# Convert into GluonTS dataset
ds = PandasDataset(dict(df))

# Split into train/test set
train, test_template = split(
    ds, offset=-TEST
)  # assign last TEST time steps as test set

# Construct rolling window evaluation
test_data = test_template.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=TEST // PDT,  # number of windows in rolling window evaluation
    distance=PDT,  # number of time steps between each window - distance=PDT for non-overlapping windows
)

# Prepare pre-trained model by downloading model weights from huggingface hub
model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-{SIZE}"),
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
)

predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data.input)

input_it = iter(test_data.input)
label_it = iter(test_data.label)
forecast_it = iter(forecasts)

inp = next(input_it)
label = next(label_it)
forecast = next(forecast_it)

plot_single(
    inp, 
    label, 
    forecast, 
    context_length=200,
    name="pred",
    show_label=True,
)
plt.show()


In [ ]:
# Convertir la serie temporal a lenguaje natural
desp = mpt.paraphrase_initial(gpt_m.name)
train_lan = mpt.paraphrase_seq2lan(train_data_months, desp)

# Realizar la predicción para los próximos 12 meses (1 año)
steps = 12  # Número de meses a predecir
model_name = 'gpt-4o-mini'  
predicted_lan = mpt.paraphrasing_predict_llm(desp, train_lan, steps, model_name)
print(predicted_lan)
# Convertir la predicción en lenguaje natural de vuelta a una serie temporal
predicted_series = mpt.recover_lan2seq(predicted_lan)

# Mostrar la serie temporal predicha
print(predicted_series)

In [3]:
from huggingface_hub import login

HF_TOKEN = os.getenv("HF_TOKEN")

#login(token=HF_TOKEN)




C:\Users\vicde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,   # usar float32 para CPU
    device_map="cpu"
)

inputs = tokenizer("¿Qué es el aprendizaje automático?", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


C:\Users\vicde\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\auto\tokenization_auto.py:1010: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [01:08<00:00, 17.13s/it]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


¿Qué es el aprendizaje automático? El aprendizaje automático (AL) es un campo de la inteligencia artificial (IA) que se enfoca en el desarrollo de algoritmos y técnicas para que las computadoras puedan aprender de los datos y mejorar su rendimiento en tareas específicas sin ser programadas explícitamente. El objetivo principal del AL es que las máquinas puedan identificar patrones y relaciones en los datos, hacer predicciones, clasificar objetos, reconocer patrones y tomar decisiones basadas en los datos de entrenamiento.
El aprendizaje automático se basa en la idea de que las computadoras pueden aprender de los datos de la misma manera que los seres humanos aprenden de la experiencia. Los algoritmos de aprendizaje automático se entrenan en grandes conjuntos de datos, que pueden incluir textos, imágenes, sonidos, etc. Una vez entrenados, los algoritmos pueden hacer predicciones o
